<a href="https://colab.research.google.com/github/Luensmann/Bachelorarbeit/blob/main/Trainer/Yepes_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate
!pip install wandb
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset, Features, Sequence, Value, ClassLabel
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, EarlyStoppingCallback, IntervalStrategy, Trainer, DataCollatorForTokenClassification
import evaluate
import json
import wandb
import os
import numpy as np


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# labels

labels = ['O', 'B-Gene_protein', 'I-Gene_protein', 'B-DNA_Mutation', 'I-DNA_Mutation', 'B-DNA_modification', 'I-DNA_modification', 'B-locus', 'I-locus', 'B-Mutation', 'I-Mutation', 'B-dbSNP', 'I-dbSNP', 'B-Protein_Mutation', 'I-Protein_Mutation', 'B-RNA', 'I-RNA', 'B-RNA_Mutation', 'I-RNA_Mutation']


In [ ]:
# Load Dataset
dataset = load_dataset("Brizape/amia_tokenized_split_0404_dev")
dataset = dataset.cast_column("ner_tags", Sequence(feature=ClassLabel(names=labels)))
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/913 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/229 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/420 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Brizape___parquet/Brizape--amia_tokenized_split_0404_dev-5c88a251eef9689d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Casting the dataset:   0%|          | 0/913 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/229 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/420 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'texts', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 913
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'texts', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 229
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'texts', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 420
    })
})

In [ ]:
tokenized_dataset = dataset.remove_columns(["id", "tokens", "ner_tags"])

In [ ]:
dataset['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-Gene_protein', 'I-Gene_protein', 'B-DNA_Mutation', 'I-DNA_Mutation', 'B-DNA_modification', 'I-DNA_modification', 'B-locus', 'I-locus', 'B-Mutation', 'I-Mutation', 'B-dbSNP', 'I-dbSNP', 'B-Protein_Mutation', 'I-Protein_Mutation', 'B-RNA', 'I-RNA', 'B-RNA_Mutation', 'I-RNA_Mutation'], id=None), length=-1, id=None),
 'texts': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [ ]:
# Tokenizer
tokenizer_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trunction=True , model_max_len=512)

In [ ]:
# padding
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
# metric
seqeval = evaluate.load("seqeval")

In [ ]:
label_list = dataset["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-Gene_protein',
 'I-Gene_protein',
 'B-DNA_Mutation',
 'I-DNA_Mutation',
 'B-DNA_modification',
 'I-DNA_modification',
 'B-locus',
 'I-locus',
 'B-Mutation',
 'I-Mutation',
 'B-dbSNP',
 'I-dbSNP',
 'B-Protein_Mutation',
 'I-Protein_Mutation',
 'B-RNA',
 'I-RNA',
 'B-RNA_Mutation',
 'I-RNA_Mutation']

In [ ]:
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

import numpy as np

labels = labels


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels, mode='strict')
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2label = {
 0: 'O',
 1: 'B-Gene_protein',
 2: 'I-Gene_protein',
 3: 'B-DNA_Mutation',
 4: 'I-DNA_Mutation',
 5: 'B-DNA_modification',
 6: 'I-DNA_modification',
 7: 'B-locus',
 8: 'I-locus',
 9: 'B-Mutation',
 10: 'I-Mutation',
 11: 'B-dbSNP',
 12: 'I-dbSNP',
 13: 'B-Protein_Mutation',
 14: 'I-Protein_Mutation',
 15: 'B-RNA',
 16: 'I-RNA',
 17: 'B-RNA_Mutation',
 18: 'I-RNA_Mutation'}

label2id = {
 'B-DNA_Mutation': 3,
 'B-DNA_modification': 5,
 'B-Gene_protein': 1,
 'B-Mutation': 9,
 'B-Protein_Mutation': 13,
 'B-RNA': 15,
 'B-RNA_Mutation': 17,
 'B-dbSNP': 11,
 'B-locus': 7,
 'I-DNA_Mutation': 4,
 'I-DNA_modification': 6,
 'I-Gene_protein': 2,
 'I-Mutation': 10,
 'I-Protein_Mutation': 14,
 'I-RNA': 16,
 'I-RNA_Mutation': 18,
 'I-dbSNP': 12,
 'I-locus': 8,
 'O': 0}

In [ ]:
# Training several model with different learning rates
ln_rate = [2e-5, 5e-5, 10e-5]

for learnrate in ln_rate:

  runname = "Yepes_" + str(learnrate) + "_0404_ES6_strict_tok"

  model = AutoModelForTokenClassification.from_pretrained(
      "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext", num_labels=19, id2label=id2label, label2id=label2id
  )

  training_args = TrainingArguments(
      report_to = 'wandb',                     # enable logging to W&B
      output_dir = runname,    # output directory/ name for huggingface hub
      learning_rate=learnrate,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      weight_decay=0.01,
      evaluation_strategy = 'steps',          # check evaluation metrics at each epoch
      max_steps = 2000,
      logging_steps = 25,                    # we will log every 25 steps
      eval_steps = 25,                      # we will perform evaluation every 25 steps
      save_steps = 25,
      load_best_model_at_end=True,
      metric_for_best_model = 'eval_loss',
      greater_is_better = False,
      push_to_hub=True,
      run_name = runname             # name of the W&B run
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_dataset["train"],
      eval_dataset=tokenized_dataset["validation"],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
      callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
  )

  trainer.train()
  wandb.finish()

  trainer.save_model()

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForToken

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-d

eval/accuracy,▁▁▁▁▁▁▁▁▁▁▁▁
eval/f1,▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,██▄▂▄▂▂▁▁▁▂▁
eval/precision,▁▁▁▁▁▁▁▁▁▁▁▁
eval/recall,▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▄▃▂▆▂▃▁█▃▄▁▅
eval/samples_per_second,▅▆▇▃▇▆█▁▆▅█▄
eval/steps_per_second,▅▆▇▃▇▆▇▁▆▅█▄
train/epoch,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇███
train/learning_rate,█▇▇▆▅▅▄▄▃▂▂▁


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/416M [00:00<?, ?B/s]

To https://huggingface.co/Brizape/Yepes_0.0001_0404_ES6_strict_tok
   5413431..737e1ce  main -> main

   5413431..737e1ce  main -> main

To https://huggingface.co/Brizape/Yepes_0.0001_0404_ES6_strict_tok
   737e1ce..00a0bed  main -> main

   737e1ce..00a0bed  main -> main



In [ ]:
wandb.finish()
trainer.save_model()

eval/accuracy,▁▁▇▇███
eval/f1,▁▂▇██▇█
eval/loss,█▃▂▁▁▂▁
eval/precision,▁▆█▇█▇█
eval/recall,▁▁▆█▇▇▇
eval/runtime,▁▃▄▆▇█▆
eval/samples_per_second,█▆▅▃▂▁▃
eval/steps_per_second,█▆▅▃▂▁▃
train/epoch,▁▁▂▂▃▃▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇███
train/learning_rate,█▇▆▅▃▂▁


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
To https://huggingface.co/Brizape/SETH_5e-05_0404_ES6_strict_tok
   b4087f7..42d5266  main -> main

   b4087f7..42d5266  main -> main

